# Einführung in pandas

pandas ist eine mächtige Bibliothek für die Arbeit mit heterogenen Daten in tabellenartiger Form. In pandas können wir Daten laden, säubern, transformieren, aggregieren und analysieren. Diese Schritte kommen vor der weiteren Verarbeitung in Machine Learning Algorithmen oder der weitergehenden Visualisierung. Das Ergebnis eines pandas-Programms ist häufig ein homogenes NumPy-Array mit numerischen Werten. Gemeinsam mit NumPy ist der array-basierte Ansatz - eine For-Schleife über die Reihen eines Datensatzes verstößt mit hoher Wahrscheinlichkeit gegen Best Practices. 

Um pandas in Python zu nutzen bietet es sich an der Standard-Konvention zu folgen und es als `pd` zu importieren:

In [ ]:
import pandas as pd

<div class="alert alert-info"><b>INFO</b>
    <p>Die Übungen und Beispiele basieren auf Daten über 
    weltweite Systeme des öffentlichen Nahverkehrs von <a href="https://www.citylines.co">https://www.citylines.co</a></p>
</div>

## pandas Series

Die grundlegende Datenstruktur in pandas ist die Series: eine eindimensionale Datenstruktur ähnlich eines eindimensionalen Arrays: 

In [ ]:
countries = pd.Series(['Sweden', 'Australia', 'Singpaore', 'Austria'])
countries

Links neben den Einträgen sehen wir den Index - da wir nichts anderes angegeben haben, werden ähnlich wie beim Array die ganzen Zahlen aufsteigend nummeriert von 0 an genommen. Mit `.values` können wir das NumPy-Array einer Series abfragen:

In [ ]:
countries.values

In [ ]:
type(countries.values)

Mit `.index` können wir auf den Index der Series zugreifen. Mit `[]` können wir per Index auf einzelne Elemente der Series zugreifen.

In [ ]:
countries.index

In [ ]:
countries.index.values

In [ ]:
countries[0]

Anders als die Indizes eines Arrays bleiben die Indizes einer Series auch bei Selektionen erhalten:

In [ ]:
countries[2:3]

In [ ]:
countries[[0,2]]

Ebenso können ähnlich wie beim `dict` andere Index-Elemente gesetzt werden, z.B.

In [ ]:
countries = pd.Series(['Sweden', 'Australia', 'Singapore', 'Austria'],
                index=['Stockholm', 'Sydney', 'Singapore', 'Vienna'])
countries

Bei nicht-numerischen Index ist die Range-Selection über `:` sowohl bei Start als auch Ende inklusive:

In [ ]:
countries['Stockholm':'Singapore']

Series können wie NumPy-Arrays miteinander kombiniert werden, dabei werden einzelne Werte über die Indizes gematcht, z.B.: 

In [ ]:
# Erstelle Series mit anderer Reihenfolge und einem Datensatz weniger
continents = pd.Series(['Europe', 'Europe', 'Asia'],
                      index=['Vienna', 'Stockholm', 'Singapore'])
# Kombiniere Series countries mit continents per String-Konkatenation
countries + ', ' + continents

Series bieten neben den aus NumPy bekannten Funktionen (z.B. `max`, `min`) noch viele weitere: https://pandas.pydata.org/docs/reference/series.html

Zum Beispiel kann mit dem `str`-Accessor auf viele String-Funktionen zurückgegriffen werden:

In [ ]:
countries.str.startswith('S')

Das Ergebnis der vorherigen Operation ist wiederum eine Series mit dem gleichen Index und Boolean-Werten. Diese Series kann nun wiederum als Index für ein Array mit gleichen Indizes verwendet werden:

In [ ]:
countries[countries.str.startswith('S')]

In [ ]:
filt_s = countries.str.startswith('S')
continents[filt_s]

## pandas DataFrame

Ein `DataFrame` bündelt mehrere `Series` in eine tabellarische Datenstruktur zusammen. Jede `Series` wird unter einem Label (Spaltenname) abgelegt und beinhaltet die Wert einer Spalte. Die Series haben im Normalfall den gleichen Index (oder zumindest überschneidend). Die Werte aus den unterschiedlichen Series mit dem gleichen Index bilden eine Reihe.
Ein DataFrame kann z.B. über ein dict erstellt werden - key ist der Spaltenname, value die Series mit den Werten:

In [ ]:
df = pd.DataFrame({"country": countries, "continent": continents})
df

Es gibt zahlreiche weitere Konstruktoren für DataFrames, z.B. basierend auf Arrays, Dicts. Siehe https://pandas.pydata.org/docs/user_guide/dsintro.html#dataframe

Wir werden meist DataFrames aus Dateien laden. Diese können in unterschiedlichen Formaten sein (z.B. csv, Excel, JSON) und sowohl auf dem lokalen Dateisystem als auch auf einem Webserver liegen. Siehe https://pandas.pydata.org/docs/user_guide/io.html

In [ ]:
# Read city data direkt von citylines.co
# cities = pd.read_csv("https://data.heroku.com/dataclips/wmeilvvkgqrderovlbhfbktsnxlm.csv")
# Wir lesen aus einer lokal gecachten Version, um den gleichen
# Datenstand für Skript und Übungen festzuhalten
cities = pd.read_csv("data/cities.csv")

In [ ]:
# Jupyer Notebook stellt DataFrames in einer gewohnten tabellarischen Ansicht dar
cities

In [ ]:
# Da DataFrames oftmals sehr viele Zeilen haben bietet sich die head-Methode an
# Sie zeigt die ersten n Zeilen an - Default-Wert ist n=5
cities.head(10)

pandas hat beim Einlesen automatisch einen numerischen Index vergeben (ganz linke Spalte ohne Spaltenname). Im Datensatz ist jedoch eine schon eine Spalte "id" vorhanden, die als Index dienen kann. Im Prinzip könnten wir auch den Namen der Städte als Index setzen, aber wenn wir die Stadttabelle mit weiteren Tabellen verknüpfen wollen wird dort die city id als Schlüssel verwendet. Mit set_index können wir eine Spalte des DataFrames als Index setzen. Wie die meisten anderen transformativen Methoden auf einem DataFrame verändert die set_index Methode standardmäßig das DataFrame nicht, sondern generiert ein neues. Dies können wir übernehmen, indem wir den inplace=True Parameter setzen oder das neue generierte DataFrame wiederum in der ursprünglichen Variable speichern.

In [ ]:
# Erzeugt ein neues DataFrame mit der Spalte id als Index
cities.set_index('id').head()

In [ ]:
# Das DataFrame cities ist unverändert:
cities.head()

In [ ]:
# Die Änderung übernehmen können wir entweder (nur eins funktioniert - danach
# ist der Index schon gesetzt und die Spalte id nicht mehr vorhanden, daher
# ist Option 1 auskommentiert
# Option 1: Abspeichern des transformierten DataFrames in der ursprünglichen Variable
# cities = cities.set_index('id')
# Option 2: inplace=True Parameter bestimmt, dass das DataFrame verändert werden soll
cities.set_index('id', inplace=True)

Beachten Sie, dass Operationen mit inplace=True keinen Rückgabewert haben. Das sehen Sie daran, dass im Jupyter Notebook nichts angezeigt wird.

## Selektionen

Auf einzelne Spalten (Series) kann per `[spalten_name]` zugegriffen werden.

In [ ]:
cities['name']

Alternativ für Namen ohne Leer- und Sonderzeichen kann auch per `.spalten_name` zugegriffen werden - wir vermeiden das im Folgenden aber Sie werden es häufig in Beispiel-Code finden.

In [ ]:
cities.name

Ähnlich wie bei Series können Selektionen mit `[]` durchgeführt werden. Da pandas hier gemäß einer Logik erschließt, ob Selektionen auf Spaltennamen, Zeilen-Labels (aus dem Index) oder Zeilen-Positionen gewünscht sind, kommt es manchmal zu unerwünschten Ergebnissen, insbesondere bei Spaltennamen die Zahlen sind. Beispiel: 

In [ ]:
# DataFrame mit Zahlen als Index-Labels und Spaltennamen
df = pd.DataFrame({0: [1, 2, 3], 1: [4, 5, 6], 2: [7, 8, 9]}, index=[1, 2, 3])
df

In [ ]:
# Zugriff auf eine Spalte mit []
df[0]

In [ ]:
# Zugriff auf Zeile per Label-Slice mit []
df[0:1]

Um Missverständnisse zu vermeiden, verwenden wir folgende Zugriffe auf ein DataFrame df:
- `df[spalten]` Zugriff auf einzelne Spalte (Series) oder Spalten (DataFrame) per Spaltenname(n)
- `df[boolean_series]` um auf alle Zeilen zuzugreifen, deren Index in `boolean_series` vorkommen und dort den Wert `True` haben
- `df.loc[zeilen]`: Zugriff auf einzelne Zeile oder Zeilen per Label(s)
- `df.loc[zeilen,spalten]`: Zugriff auf Zeilen und Spalten per Label(s) und Spaltenname(n)
- `df.iloc[zeilen]`: Zugriff auf einzelne Zeile oder Zeilen per Integer-Positionen
- `df.iloc[zeilen,spalten]`: Zugriff auf Zeilen und Spalten per Integer-Positionen


Um Spalten, Zeilen auszuwählen gibt es 3 Möglichkeiten:
- Einzelner Wert (Spaltenname, Index-Label, Integer-Position): selektiert genau diesen Wert
- Liste von Spaltennamen, Index-Labeln, Integer-Positionen: selektiert, wenn Wert in Liste
- Slices mit `start:ende`: selektiert zusammenhängenden Bereich zwischen `start` und `ende`
    - wenn `start` weggelassen wird, dann wird der erste Wert eingesetzt
    - wenn `ende` weggelassen wird, dann wird der letzte Wert eingesetzt
    - wenn beides weggelassen wird, wird der gesamte Bereich selektiert
    
Beispiele:

In [ ]:
# Einzelne Spalte als Series
cities['name']

In [ ]:
# DataFrame mit den Spalten in der Liste (in der angegeben Reihenfolge)
cities[['country', 'name']].head()

In [ ]:
# Boolean-Series (Ergebnis einer Series Operation)
cities[cities['start_year'] < 1821]

In [ ]:
# Auswahl per Label-Ranges sowohl bei Zeilen als auch Spalten
cities.loc[300:305, 'start_year':'country']

In [ ]:
# Selektiere die letzten beiden Zeilen und geraden Spaltennummern
cities.iloc[-3:-1,[0, 2, 4, 6]]

<div class="alert alert-warning">
<b>ÜBUNG:</b> DataFrames and Series
    <p>Geben Sie alle Städte aus, die (a) eine Netzlänge von über 500km haben und (b) nicht in Japan sind.
    Beachten Sie, dass die Spalten des DataFrames wiederum Series sind. Dabei sollen neben dem Stadtnamen auch die Start-Jahr, das Land und die Länge ausgegeben werden</p>
</div>

In [ ]:
#

## Werte zuweisen

Sämtliche Zugriffsarten liefern eine `View` auf das originale DataFrame. Das heißt, dass Änderungen auf der View auch im DatenFrame wiederzufinden sind. Eine Zuweisung funktioniert auch, wenn die Selektion bisher nicht im DataFrame vorhanden ist.

Beispiele:





In [ ]:
df = pd.DataFrame({'a': [1, 2, 3], 'b': [4, 5, 6], 'c': [7, 8, 9]})
df

In [ ]:
# Neue Spalte
df['d'] = [10, 11, 12]
df

In [ ]:
# Zuweisung für zweidimensional selektierten Bereich
df.loc[0:1, 'a':'b'] = [[100, 101], [102, 103]]
df

Wenn einer bestimmten Selektion ein Skalar oder ein Array von kleinerer Dimension zugewiesen wird, dann versucht pandas per `Broadcasting` die entsprechende Zuordnung vorzunehmen. Wir verwenden das vor allem, wenn wir einer Spalte einen konstanten Wert zuweisen wollen, z.B.

In [ ]:
df['e'] = 1001
df

<div class="alert alert-warning">
<b>ÜBUNG:</b> DataFrames Werte zuweisen
    <p>Fügen Sie drei neue Spalten zum cities DataFrame hinzu:
    <ol><li>age: Alter des Schienennetzes (aktuelles Jahr - start_year). Die Spalte soll direkt nach der start_year Spalte eingefügt werden. Nutzen Sie dazu die <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.insert.html">insert</a> Methode des DataFrames</li>
        <li>length_km: Länge des Liniennetzes in km statt in Metern. Nutzen Sie danach die <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html?highlight=drop#pandas.DataFrame.drop">drop</a> Methode um die Spalte length zu löschen</li>
        <li>status: 'planning', wenn die Länge 0 ist, sonst abhängig vom Startjahr 'new' (>= 2000), 'medium' (>= 1950), 'old' (ansonsten)</li><ol>
</div>

In [ ]:
#

## Beispiel Data Wrangling 

Eine Hauptaufgabe von pandas ist die Daten aus diversen Input-Formaten in das passende Output-Format zu bringen. Unser bisheriger Datensatz ist recht wohlgeformt aber in der Praxis sind die Input-Formate häufig recht schwer auslesbar und es muss mit den Daten gerangelt/gestritten werden, um sie ins richtige Format zu bringen - daher der Begriff Data Wrangling.

In unserem Datensatz müssen wir uns um die Spalte coords kümmern, die lattitude und longitude zusammengepackt in String-Form enthält. Wir werden schrittweise vorgehen:

In [ ]:
# Entfernen von POINT und den Klammern
# Der str-Accessor der coords-Series bietet entsprechende Funktionen

# Per Slice mit Integer-Positionen
cities['coords'].str.slice(6,-2)

In [ ]:
# Per Ersetzen
cities['coords'].str.replace(pat='POINT(',repl='',regex=False).str.replace(pat=')',repl='', regex=False)

In [ ]:
# Beide Ergebnisse beinhalten nun beide Koordinaten per Leerzeichen getrennt
# Hier hilft uns die str.split Methode weiter:
cities['coords'].str.slice(6,-2).str.split(' ', expand=True)

In [ ]:
# Beide Schritte inklusive Benennung der Spalten, können auch
# per Regular-Expression Matching durchgeführt werden
cities['coords'].str.extract(r'(?P<long>[-+0-9.]+) (?P<lat>[-+0-9.]+)')

Das Ergebnis der letzten Operation ist ein DataFrame mit den zwei neuen Spalten lat und long mit dem passenden Index zu unserem cities DataFrame. Um nun beide DataFrames in eins zu packen verwenden wir die <a href="https://pandas.pydata.org/docs/reference/api/pandas.concat.html">pd.concat</a> Funktion. Die Funktion hat viele Möglichkeiten, wir nutzen Sie, um zwei DataFrames "nebeneinander" zu setzen, so dass das neue DataFrame die Spalten beider DataFrames besitzt. Die Funktion verändert die übergebenen DataFrames nicht, sondern liefert ein neues DataFrame zurück.

<div class="alert alert-warning">
<b>ÜBUNG:</b> DataFrames kombinieren
    <p>
    <ol><li>Speichern Sie das Ergebnis der obigen Operation mit den extrahierten lat und long Spalten in eine neue DataFrame-Variable</li>
        <li>Wandeln Sie Datentypen der lat und long Spalten auf float</li>
        <li>Nutzen Sie pd.concat, um dieses DataFrame "rechts" an das cities DataFrame zu hängen</li>
        <li>Speichern Sie das zusammengebaute DataFrame wieder in der cities Variable</li></ol></p>
</div>

In [ ]:
#

## Daten speichern

pandas kann nicht nur Daten einlesen, sondern auch wieder in diverse Format abspeichern, siehe https://pandas.pydata.org/docs/reference/io.html

Beim Abspeichern in eine Datei sollten Sie beachten, dass existierende Dateien ohne Nachfrage überschrieben werden. Es ist immer eine gute Idee, die original Daten zu behalten, um Analysen nachvollziehbar zu machen und bei Bedarf verbessern zu können.

<div class="alert alert-warning">
<b>ÜBUNG:</b> DataFrame abspeichern
    <p>Speichern Sie Ihre Daten in eine Datei 'cities_non_zero.csv'. Es sollen folgende Daten abgespeichert werden:
    <ol><li>Nur Cities mit einer Netzlänge > 0km. Dazu können Sie die status Spalte nutzen</li>
        <li>Nur die Spalten name, country, lat, long, start_year, age, length_km - in dieser Reihenfolge</li></ol></p>
</div>

In [ ]:
#

## Abschluss

Wir haben nun erste Transformationen und Filterungen mit pandas durchgeführt und das Ergebnis in einer Datei abgespeichert. Nachdem wir die Grundlagen von pandas kennengelernt haben, werden wir uns im Folgenden eher aus Richtung der Fragestellung zu den benötigten pandas-Funktionen annähern, anstatt diese systematisch durchzugehen.